In [2]:
import pandas as pd

In [3]:
movies = pd.read_csv("../Final-project/data/movies_cleaned.csv")
actors = pd.read_csv("../Final-project/data/actors.csv")

In [19]:
merged_df = pd.merge(movies, actors, on='id', how='inner')

merged_df

,id,title_x,original_language,overview,popularity,vote_average,vote_count,credits,keywords,release_year,...,actor_1,actor_2,actor_3,actor_4,actor_5,actor_6,actor_7,actor_8,actor_9,actor_10
0,615656,Meg 2: The Trench,en,An exploratory dive into the deepest depths of...,8763.998,7.079,1365,Jason Statham-Wu Jing-Shuya Sophia Cai-Sergio ...,based on novel or book-sequel-kaiju,2023,...,Jason Statham,Wu Jing,Shuya Sophia Cai,Sergio Peris,Mencheta,Skyler Samuels,Cliff Curtis,Page Kennedy,Sienna Guillory,Melissanthi Mahut
1,758323,The Pope's Exorcist,en,Father Gabriele Amorth Chief Exorcist of the V...,5953.227,7.433,545,Russell Crowe-Daniel Zovatto-Alex Essoe-Franco...,spain-rome italy-vatican-pope-pig-possession-c...,2023,...,Russell Crowe,Daniel Zovatto,Alex Essoe,Franco Nero,Peter DeSouza,Feighoney,Laurel Marsden,Cornell John,Ryan O'Grady,Bianca Bardoe
2,667538,Transformers: Rise of the Beasts,en,When a new threat capable of destroying the en...,5409.104,7.340,1007,Anthony Ramos-Dominique Fishback-Luna Lauren V...,peru-alien-end of the world-based on cartoon-b...,2023,...,Anthony Ramos,Dominique Fishback,Luna Lauren Velez,Dean Scott Vazquez,Tobe Nwigwe,Sarah Stiles,Leni Parker,Frank Marrs,Aidan Devine,Kerwin Jackson
3,640146,Ant-Man and the Wasp: Quantumania,en,Super-Hero partners Scott Lang and Hope van Dy...,4425.387,6.507,2811,Paul Rudd-Evangeline Lilly-Jonathan Majors-Kat...,hero-ant-sequel-superhero-based on comic-famil...,2023,...,Paul Rudd,Evangeline Lilly,Jonathan Majors,Kathryn Newton,Michelle Pfeiffer,Michael Douglas,Corey Stoll,Bill Murray,William Jackson Harper,David Dastmalchian
4,677179,Creed III,en,After dominating the boxing world Adonis Creed...,3994.342,7.262,1129,Michael B. Jordan-Tessa Thompson-Jonathan Majo...,philadelphia pennsylvania-husband wife relatio...,2023,...,Michael B. Jordan,Tessa Thompson,Jonathan Majors,Wood Harris,Phylicia Rashād,Mila Davis,Kent,José Benavidez Jr.,Selenis Leyva,Florian Munteanu
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
854004,603360,An Affair,ru,A disaster film.,0.600,0.000,0,Egor Koreshkov-Mariya Kolosovskaya-Vladimir Ts...,NaN,2013,...,Egor Koreshkov,Mariya Kolosovskaya,Vladimir Tsarulkov,Alexandr Mizev,Ilya Shagalov,NaN,NaN,NaN,NaN,NaN
854005,389473,Strange Septembers: The Hill Abduction & the E...,en,The Two World-Famous UFO Experiences Took Plac...,0.600,0.000,0,NaN,NaN,2016,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
854006,461444,The Emperor's Bed,sq,In the first day of liberation partisan Meke e...,0.600,0.000,0,Pandi Raidhi,world war ii-palace-slice of life-war,1973,...,Pandi Raidhi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
854007,423700,Body Tape,en,1: Touching 2: Boxing 3: Feeling 4: Hearing...,0.600,0.000,0,NaN,NaN,1970,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
merged_df = merged_df[merged_df["vote_count"]>100] 

In [20]:
merged_df = merged_df.drop(['title_y', 'credits', 'genre_1_y', 'genre_2_y', 'genre_3_y'], axis = 1)

In [21]:
merged_df.columns
merged_df.rename(columns={'title_x': 'title'}, inplace=True)
merged_df.rename(columns={'genre_1_x': 'genre_1'}, inplace=True)
merged_df.rename(columns={'genre_2_x': 'genre_2'}, inplace=True)
merged_df.rename(columns={'genre_3_x': 'genre_3'}, inplace=True)

In [24]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.preprocessing import MultiLabelBinarizer
from scipy import sparse

# Preprocess the movie genres
merged_df['genres'] = merged_df[['genre_1', 'genre_2', 'genre_3']].astype(str).apply(' '.join, axis=1)

# Preprocess the movie actors
actors_columns = ['actor_1', 'actor_2', 'actor_3', 'actor_4', 'actor_5', 'actor_6', 'actor_7', 'actor_8', 'actor_9', 'actor_10']
merged_df['actors'] = merged_df[actors_columns].astype(str).apply(' '.join, axis=1)

# Create a TF-IDF Vectorizer for 'genres'
tfidf_genres = TfidfVectorizer(stop_words='english')
tfidf_matrix_genres = tfidf_genres.fit_transform(merged_df['genres'])

# Create a TF-IDF Vectorizer for 'actors'
tfidf_actors = TfidfVectorizer(stop_words='english')
tfidf_matrix_actors = tfidf_actors.fit_transform(merged_df['actors'])

# Combine all TF-IDF matrices
tfidf_matrix_combined = sparse.hstack([tfidf_matrix_genres, tfidf_matrix_actors])

# Compute the cosine similarity matrix
cosine_sim_combined = linear_kernel(tfidf_matrix_combined, tfidf_matrix_combined)

# Function to get recommendations based on movie indices
def get_recommendations_set(movie_indices, cosine_sim=cosine_sim_combined, merged_df=merged_df):
    # Get the pairwise similarity scores of all movies with the specified indices
    sim_scores = cosine_sim[:, movie_indices].mean(axis=1)

    # Sort the movies based on the similarity scores
    sim_scores = list(enumerate(sim_scores))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the top 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return merged_df['title'].iloc[movie_indices]

# Example: Get recommendations for a set of movies
movie_indices_set = [1, 3, 5]  # Replace with the indices of your selected movies
recommendations_set = get_recommendations_set(movie_indices_set)
print("Recommendations for the set of movies:")
print(recommendations_set)

MemoryError: Unable to allocate 4.85 TiB for an array with shape (666331374673,) and data type int64

In [10]:
def eliminar_nan(texto):
    """
    Elimina la cadena 'nan' si aparece dentro del texto.
    
    Argumentos:
    texto (str): El texto que se va a procesar.
    
    Retorna:
    str: El texto sin la cadena 'nan'.
    """
    if isinstance(texto, str):
        # Divide el texto en palabras
        palabras = texto.split()
        
        # Elimina la cadena 'nan' si está presente
        palabras_sin_nan = [palabra for palabra in palabras if palabra.lower() != 'nan']
        
        # Une las palabras nuevamente en un solo texto
        texto_limpio = ' '.join(palabras_sin_nan)
        
        return texto_limpio
    else:
        return texto

In [11]:
merged_df['keywords'].fillna('None', inplace=True)
merged_df['genres'] = merged_df['genres'].apply(lambda x: eliminar_nan(x))

In [12]:

tfidf_vectorizer_genres = TfidfVectorizer(stop_words='english')
tfidf_matrix_genres = tfidf_vectorizer_genres.fit_transform(merged_df['genres'])
tfidf_vectorizer_actors = TfidfVectorizer(stop_words='english')
tfidf_matrix_actors = tfidf_vectorizer_actors.fit_transform(merged_df['actors'])
tfidf_vectorizer_keywords = TfidfVectorizer(stop_words='english')
tfidf_matrix_keywords = tfidf_vectorizer_keywords.fit_transform(merged_df['keywords'])
tfidf_vectorizer_title = TfidfVectorizer(stop_words=None)
tfidf_matrix_title = tfidf_vectorizer_title.fit_transform(merged_df['title'])

#tfidf_vectorizer_original_language = TfidfVectorizer(stop_words=None)
#tfidf_matrix_original_language = tfidf_vectorizer_original_language.fit_transform(merged_df['original_language'])


In [13]:
from scipy.sparse import hstack

# Matrices TF-IDF
tfidf_matrices = [tfidf_matrix_genres, tfidf_matrix_actors, tfidf_matrix_keywords, tfidf_matrix_title]

# Concatenar las matrices TF-IDF horizontalmente
tfidf_matrix_combined = hstack(tfidf_matrices)

# Características numéricas
numeric_features = merged_df[['popularity', 'vote_average']].values

# Combinar las características numéricas con las matrices TF-IDF
combined_features = hstack([tfidf_matrix_combined, numeric_features])

# Ahora puedes aplicar tu algoritmo de clustering al conjunto de características combinadas


In [14]:
combined_features

<17452x67621 sparse matrix of type '<class 'numpy.float64'>'
	with 636461 stored elements in COOrdinate format>

In [15]:
from sklearn.cluster import KMeans

# Definir el número de clusters
num_clusters = 5

# Crear el objeto KMeans
kmeans = KMeans(n_clusters=num_clusters, random_state=42)

# Aplicar el algoritmo KMeans a las características combinadas
clusters = kmeans.fit_predict(combined_features)

# Asignar las etiquetas de los clústeres a las películas en merged_df
merged_df['cluster_label'] = clusters

C:\Users\polsa\AppData\Local\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [18]:
merged_df['cluster_label'].value_counts()

0    17307
4      124
3       14
1        6
2        1
Name: cluster_label, dtype: int64

In [16]:
merged_df_cluster0 = merged_df[merged_df['cluster_label'] == 0]

In [17]:
tfidf_vectorizer_genres = TfidfVectorizer(stop_words='english')
tfidf_matrix_genres = tfidf_vectorizer_genres.fit_transform(merged_df_cluster0['genres'])
tfidf_vectorizer_actors = TfidfVectorizer(stop_words='english')
tfidf_matrix_actors = tfidf_vectorizer_actors.fit_transform(merged_df_cluster0['actors'])
tfidf_vectorizer_keywords = TfidfVectorizer(stop_words='english')
tfidf_matrix_keywords = tfidf_vectorizer_keywords.fit_transform(merged_df_cluster0['keywords'])
tfidf_vectorizer_title = TfidfVectorizer(stop_words=None)
tfidf_matrix_title = tfidf_vectorizer_title.fit_transform(merged_df_cluster0['title'])

tfidf_matrices = [tfidf_matrix_genres, tfidf_matrix_actors, tfidf_matrix_keywords, tfidf_matrix_title]

# Concatenar las matrices TF-IDF horizontalmente
tfidf_matrix_combined = hstack(tfidf_matrices)

# Características numéricas
numeric_features = merged_df_cluster0[['popularity', 'vote_average']].values

# Combinar las características numéricas con las matrices TF-IDF
combined_features = hstack([tfidf_matrix_combined, numeric_features])

num_clusters = 5

# Crear el objeto KMeans
kmeans = KMeans(n_clusters=num_clusters, random_state=42)

# Aplicar el algoritmo KMeans a las características combinadas
clusters = kmeans.fit_predict(combined_features)

# Asignar las etiquetas de los clústeres a las películas en merged_df
merged_df_cluster0['cluster_label'] = clusters

C:\Users\polsa\AppData\Local\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\polsa\AppData\Local\Temp\ipykernel_10776\2954953579.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df_cluster0['cluster_label'] = clusters


In [18]:
merged_df_cluster0['cluster_label'].value_counts()

cluster_label
0    12065
4     3719
2     1059
1      362
3      108
Name: count, dtype: int64

In [ ]:
#analizar cluster en base a los generos 
#hacer una funcion que randomice peliculas para recoger el input del usuario
